## Imports

In [1]:
import numpy as np
import pandas as pd
import os
from preprocessing import Preprocessor
from dataset_balancing import Balance

Train shape : (6035, 2)
Test shape : (1065, 2)


In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Precision

## Data preprocessing

In [3]:
# Load training and test sets
df_train = pd.read_json('datas/training_set.json')
df_test = pd.read_json('datas/testing_set.json')

print(f"Train shape : {df_train.shape}")
print(f"Test shape : {df_test.shape}")

Train shape : (6035, 2)
Test shape : (1065, 2)


In [4]:
# Preprocess the data
train_preprocessor = Preprocessor(df_train)
train_preprocessor.preprocess()
test_preprocessor = Preprocessor(df_test)
test_preprocessor.preprocess()

In [ ]:
# Balance the train set
train_balance = Balance(train_preprocessor.df_dataset)
train_balance.process_balance()

In [ ]:
x_train = train_balance.df_dataset['sentence']
y_train = train_balance.df_dataset['intent']
x_test = test_preprocessor.df_dataset['sentence']
y_test = test_preprocessor.df_dataset['intent']

print("x_train.shape = ", x_train.shape)
print("y_train.shape = ", y_train.shape)
print("x_test.shape = ", x_test.shape)
print("y_test.shape = ", y_test.shape)

## Model creation and training

In [5]:
model = Sequential()
model.add(Dense(32, input_shape=(300,)))
model.add(Dense(64))
model.add(Dense(8, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                9632      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
Total params: 12,264
Trainable params: 12,264
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=[Precision()]
)

print(df_train_spacy.values)


history = model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1)

NameError: name 'df_train_spacy' is not defined